In [13]:


import hashlib
import random
import time
import glob
import os
import string
from bs4 import BeautifulSoup
import re
import numpy as np
import string
import collections
from itertools import combinations

class Shingling():
    
    shingles = []

    def k_shingle(self, text, shingle_size_k: int):
        shingles = []
        #go through text and add hashed text snippets of shingle size to array
        for index in range(len(text)-shingle_size_k+1):
            shingle=[text[index+i] for i in range(shingle_size_k)]
            shingle = ' '.join(shingle)
            shingles.append(hash(shingle))
        return list(dict.fromkeys(shingles))
    

    def __init__(self, text: str, shingle_size_k: int):
        self.shingles = self.k_shingle(text, shingle_size_k)


class CompareSets():

    jaccard_similarity = -1

    def compare(self, set1: list, set2: list):
        x = set(set1)
        y = set(set2)
        union = x.union(y)
        intersection = x.intersection(y)
        return round(len(intersection)/(len(union)), 3)
     
    def __init__(self, set1: list, set2: list):
        self.jaccard_similarity = self.compare(set1, set2)



class MinHashing():
    
    signature = None

    
    def get_signature(self, a: list, b: list, numHashes, shingleSet):
        signature = []
        Nextprime = 4294967311
        for i in range(0, numHashes):
            minHashCode = Nextprime + 1
            for shingle in shingleSet:
                hash_code = (a[i] * shingle + b[i]) % Nextprime
                if hash_code < minHashCode:
                    minHashCode = hash_code
            signature.append(minHashCode)
        return signature


    def __init__(self, a: list, b: list, numHashes: int, shingleSet):
        self.signature = self.get_signature(a, b,numHashes, shingleSet)


class CompareSignatures():
    
    estimated_similarity = -1

    def compare(self, n, set1: list, set2: list):
        c = 0
        for k in range(0, n):
            if set1[k] == set2[k]:
                c = c + 1

        return c/n

    def __init__(self, n: int, set1: list, set2: list):
        self.estimated_similarity = self.compare(n,set1, set2)


class LSH():

    candidate_pairs = []

    def lsh(self, signatures: list, sim_threshold: float):

        n_bands = 4
        n_rows = 10

        # make an empty bucket for every band
        n_buckets = 10000
        buckets = [[[] for _ in range(n_buckets)] for _ in range(n_bands)]

        for i, signature in enumerate(signatures):
            for k in range(0, n_bands):
                # concatenate and hash into a bucket
                snippet = ''.join([
                    str(x) for x in signature[k * n_rows:k * n_rows + n_rows]
                ])
                bucket = hash(snippet) % n_buckets
                buckets[k][bucket].append(i + 1)

        # clear buckets of duplicates
        for i, bucket in enumerate(buckets):
            for k, element in enumerate(bucket):
                element = list(dict.fromkeys(element))

        # make sure that buckets have more than 1 signature
        for i, bucket in enumerate(buckets):
            if len(bucket) < 2:
                buckets.pop(i)

        possible_pairs = []
        # make all possible pairs
        for bucket_band in buckets:
            for bucket in bucket_band:
                pairs = [i for i in combinations(bucket, 2)]
                possible_pairs += pairs

        c = collections.Counter(possible_pairs)
        hits = []
        for i, value in enumerate(c.values()):
            if (value / n_bands) >= sim_threshold**n_rows:
                hits.append(i)
        candidate_pairs = []
        for i, pair in enumerate(c.keys()):
            if i in hits:
                candidate_pairs.append(pair)

        return candidate_pairs

    def __init__(self, signatures: list, sim_threshold: float):
        self.candidate_pairs = self.lsh(signatures, sim_threshold)

def get_random_coeff(shingle_size_k):
    maxShingleId = 2**32 - 1
    randomList = []
    while shingle_size_k > 0:
        randomIndex = random.randint(0, maxShingleId)
        while randomIndex in randomList:
            randomIndex = random.randint(0, maxShingleId)
        randomList.append(randomIndex)
        shingle_size_k = shingle_size_k - 1
    return randomList

def main():   
    
    # config
    
    threshold = 0.8
    shingle_size_k = 5
    num_docs = 100
    print('For {} documents the execution times are:'.format(num_docs))
    
    
    ##################
    #  Prepare Data  #  
    ##################
    
    data = []
  
    #extract text from dataset
    for file in os.listdir("data/"):
        if file.endswith(".sgm"):
            filename = os.path.join("data", file)
            f = open(filename, 'r', encoding='utf-8', errors='ignore')
            dataFile = f.read()
            soup = BeautifulSoup(dataFile, 'html.parser')
            contents = soup.findAll('body')
            for content in contents:
                data.append(content.text)

    #make text lowercase and remove punctuation
    for i in data:
        i = i.lower()
        i = i.translate(str.maketrans('','',string.punctuation))
    

    
    data= list(map(lambda text: re.sub(r'http\S+', '', text, flags=re.MULTILINE), data))
    
    def removeHTMLTags(text):
        text = re.sub(r'<.*?>', '', text, flags=re.MULTILINE)
        return text
    
    
    for i in range(100):
        data[i]=data[i].lower()
        data[i] = data[i].translate(str.maketrans('','',string.punctuation))

    data= list(map(removeHTMLTags, data))
    
    
    
    #######################
    #  Shingle Documents  #  
    #######################

    
    print('Shingling')
    shingled_documents = []
    begin = time.time()
    for i in range(num_docs):
        shingled_documents.append(Shingling(data[i], shingle_size_k))
        
    for i in range(len(shingled_documents)-1):
        for j in range(i+1, len(shingled_documents)):
            similarity = CompareSets(shingled_documents[i].shingles, shingled_documents[j].shingles).jaccard_similarity
            if similarity > threshold:
                print('Jaccard similarity between document {} and {} is {}'.format(i, j, similarity))
    end = time.time()
    print('Execution time for Shingling is {} seconds'.format(round(end-begin , 2)))
    
    
    ######################################################
    #  MinHash Shingles and calculate Jaccard Similarity #  
    ######################################################
    
    print('MinHashing')
    begin = time.time()
    NumHash = 30
    a = get_random_coeff(NumHash)           
    b = get_random_coeff(NumHash)
    
    minhash_documents = []
    for i in range(num_docs):
        minhash_documents.append(MinHashing(a, b,NumHash, shingled_documents[i].shingles))
    for i in range(len(minhash_documents)-1):
        for j in range(i+1, len(minhash_documents)):
            similarity = CompareSignatures(NumHash,minhash_documents[i].signature, minhash_documents[j].signature).estimated_similarity
            if similarity > threshold:
                print('Jaccard similarity between document {} and {} is {}'.format(i, j, similarity))
    end = time.time()
    print('Execution time for MinHashing is {} seconds'.format(round(end-begin, 3)))
    
    print('LSH')
    begin = time.time()
    k=[i.signature for i in minhash_documents]
    lsh = LSH(k, 0.8)
    end = time.time()
    print('Execution time for LSH: {} seconds'.format(round(end-begin, 3)))

main()

 



For 100 documents the execution times are:
Shingling
Jaccard similarity between document 13 and 33 is 0.808
Jaccard similarity between document 31 and 34 is 1.0
Jaccard similarity between document 46 and 59 is 0.979
Jaccard similarity between document 57 and 62 is 1.0
Execution time for Shingling is 0.45 seconds
MinHashing
Jaccard similarity between document 13 and 33 is 0.8666666666666667
Jaccard similarity between document 31 and 34 is 1.0
Jaccard similarity between document 46 and 59 is 1.0
Jaccard similarity between document 57 and 62 is 1.0
Execution time for MinHashing is 0.581 seconds
LSH
Execution time for LSH: 0.147 seconds
